# Similar Abstract Search

https://medium.com/@davidlfliang/intro-getting-started-with-text-embeddings-using-bert-9f8c3b98dee6

FAISS

https://medium.com/loopio-tech/how-to-use-faiss-to-build-your-first-similarity-search-bf0f708aa772

In [ ]:
# Use kaggle API to download example data. arxiv apaper Abstracts and there is also arXive Dataset (4.5 GB)
import kagglehub

# Download latest version
# path = kagglehub.dataset_download("Cornell-University/arxiv")

# print("data/arxiv_dataset", path)

/home/piotr/projects/articles_similarity/scientific_abstracts_similarity/dev/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 1.37G/1.37G [04:39<00:00, 5.25MB/s]

Extracting files...


data/arxiv_dataset /root/.cache/kagglehub/datasets/Cornell-University/arxiv/versions/208


In [ ]:
# !cp /root/.cache/kagglehub/datasets/Cornell-University/arxiv/versions/208/arxiv-metadata-oai-snapshot.json /home/piotr/projects/articles_similarity/scientific_abstracts_similarity/data/arxiv_papers

## Preprocessing

### Tokenization

In [ ]:
# from transformers import BertTokenizer

# bert_tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

# text = "The quick brown fox jumps over the lazy dog."

# bert_inputs = bert_tokenizer(text, return_tensors='pt')

# print('Token IDs: ', bert_inputs['input_ids'])

# attention_mask = bert_inputs['attention_mask']
# print('Attention Mask: ', attention_mask)

# token_type_ids = bert_inputs['token_type_ids']
# print('Token Type IDs:', token_type_ids)

# tokens = bert_tokenizer.convert_ids_to_tokens(bert_inputs['input_ids'][0])
# print('Tokens:', tokens)

Token IDs:  tensor([[  101,  1109,  3613,  3058, 17594, 15457,  1166,  1103, 16688,  3676,
           119,   102]])
Attention Mask:  tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
Token Type IDs: tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
Tokens: ['[CLS]', 'The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog', '.', '[SEP]']


In [ ]:
# bert_inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

## Embedding Generation

In [ ]:
# from transformers import BertModel
# import torch

# model = BertModel.from_pretrained(MODEL_NAME)

# with torch.no_grad():
#     outputs = model(**bert_inputs)

# last_hidden_states = outputs.last_hidden_state

# print('Shape of the last hidden state (embedding):', last_hidden_states.shape)

# tokens = bert_tokenizer.convert_ids_to_tokens(bert_inputs['input_ids'][0])
# for token, embedding in zip(tokens, last_hidden_states[0]):
#     print(f"Token: {token}, Embedding Dimension:{embedding.shape},\nEmbedding (first 5 components): {embedding[:5]}")


Shape of the last hidden state (embedding): torch.Size([1, 12, 1024])
Token: [CLS], Embedding Dimension:torch.Size([1024]),
Embedding (first 5 components): tensor([-0.3982, -0.5164,  0.1184,  0.2094, -0.0663])
Token: The, Embedding Dimension:torch.Size([1024]),
Embedding (first 5 components): tensor([-0.5459, -0.9618,  0.1409, -0.1374, -0.1215])
Token: quick, Embedding Dimension:torch.Size([1024]),
Embedding (first 5 components): tensor([-0.1578,  0.3739, -0.2527, -0.0500,  0.7173])
Token: brown, Embedding Dimension:torch.Size([1024]),
Embedding (first 5 components): tensor([-1.0575, -0.4547,  0.0686, -0.6760,  0.7317])
Token: fox, Embedding Dimension:torch.Size([1024]),
Embedding (first 5 components): tensor([-0.1732, -0.2297, -0.7842,  0.0217,  0.0425])
Token: jumps, Embedding Dimension:torch.Size([1024]),
Embedding (first 5 components): tensor([-0.5367,  0.0404, -0.0642, -0.2768,  0.2793])
Token: over, Embedding Dimension:torch.Size([1024]),
Embedding (first 5 components): tensor([-

Models like SBERT (Sentence-BERT) are specialized in generating high-quality sentence embeddings.

## Text Similarity Search

### Handle 4.5 GB JSON file

In [2]:
MODEL_NAME = 'bert-base-uncased'
dataset_path = '../data/arxiv_papers/arxiv-metadata-oai-snapshot.json'

Read one example line

In [3]:
import json

# # Open the NDJSON file
# with open(dataset_path, 'r') as f:
#     # Read the first line
#     line = f.readline().strip()
#     # Parse the JSON object
#     data = json.loads(line)
#     print(data)  # Output the parsed JSON


Read lines from the file.

In [4]:
texts = []
with open(dataset_path, 'r') as f:
    for line in f:
        data = json.loads(line)
        texts.append(data)
        print(data)
        if len(texts) > 9:
            break

{'id': '0704.0001', 'submitter': 'Pavel Nadolsky', 'authors': "C. Bal\\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan", 'title': 'Calculation of prompt diphoton production cross sections at Tevatron and\n  LHC energies', 'comments': '37 pages, 15 figures; published version', 'journal-ref': 'Phys.Rev.D76:013009,2007', 'doi': '10.1103/PhysRevD.76.013009', 'report-no': 'ANL-HEP-PR-07-12', 'categories': 'hep-ph', 'license': None, 'abstract': '  A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with data from the Fermilab

## Create Embeddings

In [1]:
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import StringType, ArrayType
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from pyspark.sql.functions import col

MODEL_NAME = 'bert-base-uncased'
dataset_path = '../data/arxiv_papers/arxiv-metadata-oai-snapshot.json'

spark = SparkSession.builder \
    .appName("Embeddings for Abstracts") \
    .config("spark.driver.memory", "15g") \
    .config("spark.executor.memory", "30g") \
    .config("spark.sql.autoBroadcastJoinThreshold", "1000MB") \
    .getOrCreate()
    
df = spark.read.json(dataset_path)

/home/piotr/projects/articles_similarity/scientific_abstracts_similarity/dev/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
your 131072x1 screen size is bogus. expect trouble
24/12/09 11:41:37 WARN Utils: Your hostname, Piotr-Szefer resolves to a loopback address: 127.0.1.1; using 172.20.255.242 instead (on interface eth0)
24/12/09 11:41:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/09 11:41:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_small = df.limit(2)
df_small.show()
df_small.printSchema()

+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+---------+--------------------+--------------------+----------------+--------------+--------------------+-----------+--------------------+
|            abstract|             authors|      authors_parsed|   categories|            comments|                 doi|       id|         journal-ref|             license|       report-no|     submitter|               title|update_date|            versions|
+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+---------+--------------------+--------------------+----------------+--------------+--------------------+-----------+--------------------+
|  A fully differe...|C. Bal\'azs, E. L...|[[Balázs, C., ], ...|       hep-ph|37 pages, 15 figu...|10.1103/PhysRevD....|0704.0001|Phys.Rev.D76:0130...|                NULL|ANL-HEP-PR-07-12|Pavel Nadolsky|Calculation of pr..

In [ ]:
# @pandas_udf(StringType())
# def get_string_as_input_udf(text):
#     # print(text)
#     # text = text.to_string()
#     output = []
#     for txt in text:
#         # print(f'Loop: {txt}')
#         # print(type(txt))
#         output.append(txt.capitalize())
#     # print(f'Output: {output} of length {len(output)}')
#     return pd.Series(output)

# res_df = df_small.withColumn(
#     'capitalized', get_string_as_input_udf(df_small.abstract)
#     )

# res_df.show()


+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+---------+--------------------+--------------------+----------------+--------------+--------------------+-----------+--------------------+--------------------+--------------------+
|            abstract|             authors|      authors_parsed|   categories|            comments|                 doi|       id|         journal-ref|             license|       report-no|     submitter|               title|update_date|            versions|  abstract_as_string|         capitalized|
+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+---------+--------------------+--------------------+----------------+--------------+--------------------+-----------+--------------------+--------------------+--------------------+
|  A fully differe...|C. Bal\'azs, E. L...|[[Balázs, C., ], ...|       hep-ph|37 pages, 15 figu..

In [ ]:
from pyspark.sql.types import FloatType

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = BertModel.from_pretrained(MODEL_NAME)

@pandas_udf(ArrayType(FloatType()))
def get_sentence_embedding_udf(texts):
    
    embeddings = []

    for text in texts:
        print(f'[INFO] pass {type(text)} to tokenizer.')
        inputs = tokenizer(text, return_tensors='pt')
    
        # print(f'[INFO] pass {inputs} to model.')
        with torch.no_grad():
            outputs = model(**inputs)
    
        last_hidden_states = outputs.last_hidden_state
        embedding = torch.mean(last_hidden_states, dim=1).numpy()
        print(f'Type of the last hidden state is: {type(embedding[0])}')
        embeddings.append(embedding[0])
        print(f'An embed: {embedding[0]} appended to the list...')

    return pd.Series(embeddings)

result_df = df_small.withColumn(
    'embedding_bert', 
    get_sentence_embedding_udf(df_small.abstract)
    )

result_df.show()

[INFO] pass <class 'str'> to tokenizer.                             (0 + 1) / 1]
Type of the last hidden state is: <class 'numpy.ndarray'>
An embed: [-3.77148062e-01 -4.81337532e-02  3.44414890e-01 -1.93679184e-01
  5.33446781e-02 -9.25121233e-02 -1.14319779e-01  8.87638927e-02
  5.33394441e-02 -1.04521960e-01 -2.46438831e-02 -3.02394837e-01
 -4.31008786e-01  3.86774033e-01 -1.23301834e-01  6.62005246e-01
  3.10081750e-01  8.59795697e-03 -2.74683069e-02  5.85905872e-02
  5.39790928e-01  1.74066707e-01 -1.57918602e-01  6.68664217e-01
  3.02956283e-01 -3.70178968e-01 -4.09604013e-02  4.46310453e-02
  2.86020376e-02 -6.92848638e-02  1.29051879e-01  3.55074942e-01
 -3.53719413e-01 -2.33326599e-01  2.81835616e-01  2.32169285e-01
  3.27921920e-02 -1.87464550e-01  3.80876623e-02  1.71333537e-01
 -5.31015456e-01 -6.08658612e-01 -1.01786986e-01  7.40590021e-02
 -4.92804289e-01 -4.31992143e-01  4.44798827e-01 -1.99442729e-02
  4.17272262e-02 -2.48743445e-01 -5.55935979e-01  6.77505881e-02
  1.15

+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+---------+--------------------+--------------------+----------------+--------------+--------------------+-----------+--------------------+--------------------+
|            abstract|             authors|      authors_parsed|   categories|            comments|                 doi|       id|         journal-ref|             license|       report-no|     submitter|               title|update_date|            versions|      embedding_bert|
+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+---------+--------------------+--------------------+----------------+--------------+--------------------+-----------+--------------------+--------------------+
|  A fully differe...|C. Bal\'azs, E. L...|[[Balázs, C., ], ...|       hep-ph|37 pages, 15 figu...|10.1103/PhysRevD....|0704.0001|Phys.Rev.D76:0130...|         

In [ ]:
# tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
# model = BertModel.from_pretrained(MODEL_NAME)

# @pandas_udf(StringType())
# def get_sentence_embedding_udf(text):
    
#     text = text.to_string()

#     print(f'[INFO] pass {type(text)} to tokenizer.')
#     inputs = tokenizer(text, return_tensors='pt')
    
#     print(f'[INFO] pass {inputs} to model.')
#     with torch.no_grad():
#         outputs = model(**inputs)
    
#     last_hidden_states = outputs.last_hidden_state
#     sentence_embedding = torch.mean(last_hidden_states, dim=1).numpy()
    
#     return sentence_embedding

# result_df = df_small.withColumn(
#     'embedding_bert', 
#     get_sentence_embedding_udf(df_small.abstract.cast('string'))
#     )

# result_df.show()

In [18]:
spark.stop()

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
embeddings = [get_sentence_embedding(text['abstract']) for text in texts]
query_embedding = embeddings[0]
similarities = cosine_similarity(query_embedding, np.vstack(embeddings[1:]))
print(sorted(similarities[0], reverse=True))
# Print query text
print (f"Query text: {texts[0]['title']}")

# Print similarities
for i, text in enumerate(texts[1:]):
    print(f"Similarity with '{text['title']}': {similarities[0][i]}")